## CAI - Rebalancing data for December 2023 

## Configuring packages

In [35]:
import pandas as pd
import numpy as np
import requests
import decouple
import sys 
import datetime
import time
sys.path.append('../')

from web3 import Web3
from abis import index_anatomy


infura_url = decouple.config("AVALANCHE_INFURA_URL")
web3_object = Web3(Web3.HTTPProvider(infura_url))

import db_funcs

In [36]:
from pycoingecko import CoinGeckoAPI
key = decouple.config("CG_KEY")
cg = CoinGeckoAPI(api_key= key)

In [37]:
import plotly.express as px
import plotly.graph_objects as go

In [38]:
from plotly.offline import plot, iplot, init_notebook_mode
init_notebook_mode(connected=True)

## Index Variables and settings

In [39]:
db_key_date = '2023-12-01'
db_liquidity_table = 'cai_liquidities'
db_benchmark_table = 'cai_benchmark_data'
min_weight = 0.005
max_weight = 0.5
supply_threshold = 0.3
liquidity_consistency = 90 # In days
avax_to_savax_split = 0.6
liveness_threshold = 30*6 # In days

In [40]:
liq = 2*10**6
liq_os = liq / 2 # one-side liquidity 
trade_value_tiny = 1e2
trade_value = 1e5
max_slippage = (liq_os + trade_value_tiny) / (liq_os + trade_value) - 1
# round to the nearest .5
max_slippage = round(2 * max_slippage, 2) / 2 
max_slippage

-0.09

## Data retrieving and filtering

### Token inclusion criteria

1. **Project and token characteristics**
    1. The project’s token should have been listed on CoinGecko with pricing data at least 6 months prior to the date of inclusion in the index.
    1. The project should have a token that is native to Avalanche. This excludes wrapped variants, where the underlying tokens are locked on an alt-L1.
    1. The project should be a going concern, with a dedicated team actively building, supporting and maintaining the project.
    1. No rebasing or deflationary tokens.
    1. The project must be widely considered to be building a useful protocol or product. Projects that have ponzi characteristics at the core of their offering will not be considered.
    1. Synthetic tokens which derive their value from external price feeds are not permissible.
    1. The project’s token must not have the ability to pause token transfers.
    1. The project’s protocol or product must have significant usage.
    <br>
1. **Pricing requirements**
    1. Token should have a Chainlink price feed to allow for manipulation resistant pricing.
    <br>
1. **Token Supply requirements**
    1. The project's token must have a circulating supply greater than 30% of the max supply. In cases where a token does not have a max supply, the minting mechanics would need to be assessed.
    <br>
1. **Liquidity Requirements**
    1. The token must be listed on a supported exchange. 
    1. The token should have in aggregate at least $2mm of onchain liquidity across Trader Joe, Platypus, Pangolin.
    1. The token must have shown consistent DeFi liquidity on Avalanche.
    <br>
1. **Security Requirements**
    1. The project must have been audited by smart contract security professionals with the audit report(s) publicly available. Alternatively, the protocol must have been operating long enough to create a consensus about its safety in the decentralised finance community.

### Tokens list

In [41]:
def normalize_symb(symb):
    return ''.join(c for c in symb if c.isalnum()).upper()

#### CoinGecko

##### Tokens addresses

In [42]:
coins_list = cg.get_coins_list(include_platform=True)
coins = {}
for coin in coins_list:
    if 'avalanche' in coin['platforms']:
        coin['addr'] = coin['platforms']['avalanche']
        coin['symbol'] = normalize_symb(coin['symbol'])
        coins[coin['id']] = coin

# Adding AVAX
coins['avalanche-2'] = {'id': 'avalanche-2','symbol':normalize_symb('avax'), 'name':'Avalanche','addr':'0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7',}


##### Market data

In [43]:
cat_id = "avalanche-ecosystem"
cg_tokens_data = cg.get_coins_markets(vs_currency='USD', order='market_cap_desc', category=cat_id)
for i in range(2, 30):
    cg_tokens_data.extend(cg.get_coins_markets(vs_currency='USD', order='market_cap_desc', category=cat_id, page=i))
    
cg_tokens_data = pd.DataFrame.from_records(cg_tokens_data)


#print(cg.get_coins_markets(ids='avalanche-2',vs_currency='USD'))

cg_tokens_data = pd.concat([cg_tokens_data,pd.DataFrame(cg.get_coins_markets(ids='avalanche-2',vs_currency='USD'))],ignore_index=True)
cg_tokens_data

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,total_supply,max_supply,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated
0,tether,usdt,Tether,https://assets.coingecko.com/coins/images/325/...,1.000000,8.965243e+10,3.0,8.981896e+10,7.032443e+10,1.004,...,8.981412e+10,NaN,1.32,-24.41558,2018-07-24T00:00:00.000Z,0.572521,74.67550,2015-03-02T00:00:00.000Z,None,2023-12-04T23:40:01.693Z
1,binancecoin,bnb,BNB,https://assets.coingecko.com/coins/images/825/...,232.900000,3.583588e+10,4.0,3.583588e+10,2.317277e+09,235.570,...,1.538562e+08,2.000000e+08,686.31,-66.06218,2021-05-10T07:24:17.097Z,0.039818,584859.74692,2017-10-19T00:00:00.000Z,None,2023-12-04T23:42:47.950Z
2,usd-coin,usdc,USDC,https://assets.coingecko.com/coins/images/6319...,0.999718,2.435866e+10,7.0,2.435192e+10,1.662824e+10,1.002,...,2.437277e+10,NaN,1.17,-14.73072,2019-05-08T00:40:28.300Z,0.877647,13.93629,2023-03-11T08:02:13.981Z,None,2023-12-04T23:42:49.531Z
3,chainlink,link,Chainlink,https://assets.coingecko.com/coins/images/877/...,15.720000,8.747675e+09,12.0,1.570921e+10,1.054547e+09,16.350,...,1.000000e+09,1.000000e+09,52.70,-70.18773,2021-05-10T00:13:57.214Z,0.148183,10501.77609,2017-11-29T00:00:00.000Z,None,2023-12-04T23:42:49.917Z
4,avalanche-2,avax,Avalanche,https://assets.coingecko.com/coins/images/1255...,22.400000,8.175898e+09,13.0,9.698908e+09,6.807323e+08,23.120,...,4.332507e+08,7.200000e+08,144.96,-84.52840,2021-11-21T14:18:56.538Z,2.800000,700.68907,2020-12-31T13:15:21.540Z,None,2023-12-04T23:42:54.631Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
352,indestcoin,intd,INDESTCOIN,https://assets.coingecko.com/coins/images/2754...,NaN,NaN,NaN,NaN,NaN,NaN,...,1.000000e+10,1.000000e+10,NaN,0.00000,None,NaN,0.00000,None,None,None
353,aave-v3-savax,asavax,Aave v3 sAVAX,https://assets.coingecko.com/coins/images/3291...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.00000,None,NaN,0.00000,None,None,None
354,aave-v3-wavax,awavax,Aave v3 WAVAX,https://assets.coingecko.com/coins/images/3291...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.00000,None,NaN,0.00000,None,None,None
355,aave-v3-btc-b,abtc.b,Aave v3 BTC.b,https://assets.coingecko.com/coins/images/3291...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.00000,None,NaN,0.00000,None,None,None


In [44]:
tokens_market_data = cg_tokens_data.set_index('id')[
    ['name', 'symbol', 'market_cap', 'market_cap_rank', 'current_price', 'total_supply', 'circulating_supply']
]
tokens_market_data['symbol'] = tokens_market_data['symbol'].apply(normalize_symb)

In [45]:
tokens_market_data = tokens_market_data[tokens_market_data['market_cap'] > 0]
tokens_market_data = tokens_market_data[~tokens_market_data.index.duplicated(keep='first')]
tokens_market_data = tokens_market_data.join(pd.DataFrame.from_dict(coins, orient='index')['addr'])
tokens_market_data = tokens_market_data[~tokens_market_data['addr'].isna()]


In [46]:
# Stable coins to remove from tokens list
stablecoins = pd.DataFrame(cg.get_coins_markets('usd',category='stablecoins')).set_index('id')

#Adjusting Pangolin's max supply to reflect tokenomic changes here: https://app.pangolin.exchange/#/vote/10
tokens_market_data.loc[['pangolin'],["total_supply"]] = 230000000.0

tokens_market_data =  tokens_market_data[~tokens_market_data.index.isin(stablecoins.index)]
tokens_market_data = tokens_market_data.sort_values('market_cap', ascending=False)

tokens_market_data

,name,symbol,market_cap,market_cap_rank,current_price,total_supply,circulating_supply,addr
id,,,,,,,,
chainlink,Chainlink,LINK,8.747675e+09,12.0,15.720000,1.000000e+09,5.568500e+08,0x5947bb275c521040051d82396192181b413227a3
avalanche-2,Avalanche,AVAX,8.175898e+09,13.0,22.400000,4.332507e+08,3.652178e+08,0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7
wrapped-bitcoin,Wrapped Bitcoin,WBTC,6.688288e+09,16.0,41804.000000,1.602864e+05,1.602864e+05,0x50b7545627a5162f82a992c33b87adc75187b218
uniswap,Uniswap,UNI,4.547819e+09,21.0,6.080000,1.000000e+09,7.537667e+08,0x8ebaf22b6f053dffeaf46f4dd9efa95d89ba8580
aave,Aave,AAVE,1.462220e+09,47.0,99.740000,1.600000e+07,1.466432e+07,0x63a72806098bd3d9520cc43356dd78afe5d386d9
...,...,...,...,...,...,...,...,...
nftearth,NFTEarth,NFTE,1.146627e+04,3674.0,0.000115,1.000000e+08,1.000000e+08,0xd47e4f1ef1aa4090bc27420bdd5cb379ced81440
ape-in,Ape In,APEIN,5.793130e+03,3715.0,0.154483,3.750000e+04,3.750000e+04,0x938fe3788222a74924e062120e7bfac829c719fb
echidna,Echidna,ECD,4.576470e+03,3726.0,0.000398,3.385084e+08,1.149388e+07,0xeb8343d5284caec921f035207ca94db6baaacbcd


### Token's supply check 

In [47]:
sp_passed_index = (tokens_market_data['circulating_supply'] / tokens_market_data['total_supply']) > supply_threshold
sp_passed = tokens_market_data[sp_passed_index]

### Retrieve historical data & token age check

In [48]:
exclude_list = []

prices_data = pd.DataFrame()
marketcaps = pd.DataFrame() 
for id_, row in sp_passed.iterrows():
    symbol = row['symbol']
    data = cg.get_coin_market_chart_by_id(id_, vs_currency='USD', days='max')
  
    df_prices = pd.DataFrame(data['prices'], columns=['date', id_])
    df_prices = df_prices[df_prices[id_] > 0]
    df_prices['date'] = pd.to_datetime(df_prices['date'], unit='ms').dt.date
    df_prices['date'] = pd.to_datetime(df_prices['date'])
    df_prices = df_prices.set_index('date', drop=True)
    df_prices = df_prices.loc[~df_prices.index.duplicated(keep='first')]
    
    if len(df_prices) < liveness_threshold:
        print(f'Excluding {symbol}, prices data available only for {len(df_prices)} < {liveness_threshold} days')
        exclude_list.append(id_)
        continue
    prices_data = pd.concat([prices_data, df_prices], axis=1)
    
    df_mcaps = pd.DataFrame(data['market_caps'], columns=['date', id_])
    df_mcaps = df_mcaps[df_mcaps[id_] > 0]
    df_mcaps['date'] = pd.to_datetime(df_mcaps['date'], unit='ms').dt.date
    df_mcaps['date'] = pd.to_datetime(df_mcaps['date'])
    df_mcaps = df_mcaps.set_index('date', drop=True)
    df_mcaps = df_mcaps.loc[~df_mcaps.index.duplicated(keep='first')]
    
    if len(df_mcaps) < liveness_threshold:
        print(f'Note: {symbol}, marketcap data available only for {len(df_mcaps)} < {liveness_threshold} days')
        ## continue
    df_mcaps = df_mcaps.reindex(df_prices.index)
    marketcaps = pd.concat([marketcaps, df_mcaps], axis=1)
    
    

Excluding 3ULL, prices data available only for 179 < 180 days
Excluding USP, prices data available only for 13 < 180 days
Note: ERN, marketcap data available only for 146 < 180 days
Note: EUROE, marketcap data available only for 148 < 180 days
Note: OBX, marketcap data available only for 89 < 180 days
Note: VCHF, marketcap data available only for 66 < 180 days
Note: EIV, marketcap data available only for 86 < 180 days
Note: NFTE, marketcap data available only for 73 < 180 days


### Non quantative checks

In [49]:
manual_exclusions = [
    {
        'gecko_id': 'spell-token',
        'reason': 'Not a native Avalanche asset'
    },
    {
        'gecko_id': 'stacktical',
        'reason': 'Not a native Avalanche asset'
    },
    {
        'gecko_id': 'zookeeper',
        'reason': 'Not a native Avalanche asset'
    },
    {
        'gecko_id': 'pendle',
        'reason': 'Not a native Avalanche asset'
    },
    {
        'gecko_id': 'everrise',
        'reason': 'Not a native Avalanche asset'
    },
    {
        'gecko_id': 'wrapped-bitcoin',
        'reason' : 'Not a native Avalanche asset'
    },
    {
        'gecko_id': 'bitcoin-avalanche-bridged-btc-b',
        'reason': 'Not a native Avalanche asset'
    },
    {
        'gecko_id': 'gmx',
        'reason': 'Bridged asset'
    },
    {
        'gecko_id': 'woo-network',
        'reason': 'Bridged asset'
    },
    {
        'gecko_id': 'euro-coin',
        'reason': 'Stable coin'
    }
]
for token in manual_exclusions:
    exclude_list.append(token['gecko_id'])

In [50]:
passed_q_and_non_q_checks = sp_passed.query('index not in @exclude_list')
passed_q_and_non_q_checks.drop(passed_q_and_non_q_checks[passed_q_and_non_q_checks['market_cap'] < 1e6].index)

,name,symbol,market_cap,market_cap_rank,current_price,total_supply,circulating_supply,addr
id,,,,,,,,
chainlink,Chainlink,LINK,8.747675e+09,12.0,1.572000e+01,1.000000e+09,5.568500e+08,0x5947bb275c521040051d82396192181b413227a3
avalanche-2,Avalanche,AVAX,8.175898e+09,13.0,2.240000e+01,4.332507e+08,3.652178e+08,0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7
uniswap,Uniswap,UNI,4.547819e+09,21.0,6.080000e+00,1.000000e+09,7.537667e+08,0x8ebaf22b6f053dffeaf46f4dd9efa95d89ba8580
aave,Aave,AAVE,1.462220e+09,47.0,9.974000e+01,1.600000e+07,1.466432e+07,0x63a72806098bd3d9520cc43356dd78afe5d386d9
the-graph,The Graph,GRT,1.392043e+09,50.0,1.493450e-01,1.000000e+10,9.321063e+09,0x8a0cac13c7da965a312f08ea4229c37869e85cb9
...,...,...,...,...,...,...,...,...
vector-finance,Vector Finance,VTX,1.209312e+06,2135.0,6.421300e-02,1.887405e+07,1.887405e+07,0x5817d4f0b62a59b17f75207da1848c2ce75e7af4
shibavax,Shibavax,SHIBX,1.110644e+06,2176.0,1.141900e-04,1.000000e+10,9.723385e+09,0x440abbf18c54b2782a4917b80a1746d3a2c2cce1
cosmic-universe-magic-token,Cosmic Universe Magic,MAGIC,1.064734e+06,2194.0,1.664043e-02,6.501269e+07,6.389206e+07,0x9a8e0217cd870783c3f2317985c57bf570969153


### Liquidity check

To derive a slippage threshold, consider an abstract univ2-like pool with $5m\$$ overall liquidity. 
I.e if pool has reserves $R_1$, $R_2$ and $p_1$ and $p_2$ are the correspondent tokens prices (in USD), then
$$R_1p_1 = R_2p_2 = 2.5m \$$$

Swapping $x$ token1 for token2 will result in the output amount $y$ such that
$$R_1\cdot R_2 = (R_1 + x) \cdot (R_2 - y)$$
$$y = \frac{R_2\cdot x}{R_1 + x}$$
Then the delivery price will be
$$p_d = \frac{y}{x} = \frac{R_2}{R_1 + x}$$
The spot price may be estimated as a delivery price for relatively small input amount $x_0$
$$p_s = \frac{y_0}{x_0} = \frac{R_2}{R_1 + x_0}$$

Then the theoretical slippage for this pool will be
$$slippage = \frac{p_d}{p_s} - 1 = \frac{R_1+x_0}{R_1+x} - 1 = \frac{p_1R_1+p_1x_0}{p_1R_1+p_1x}-1$$

In [51]:
url_0x = "https://avalanche.api.0x.org/swap/v1/quote"
header =  {'0x-api-key': decouple.config("ZEROEX_KEY")}

In [52]:
decimals = 18
slippages = []
for id, data in passed_q_and_non_q_checks.iterrows():
    try:
        query = {
            'buyToken': data['addr'],
            'sellToken': 'AVAX',
            'sellAmount': int(trade_value_tiny / tokens_market_data.loc['avalanche-2']['current_price'] * 10 ** decimals),
            'enableSlippageProtection':'true'
        }
        
        # spot price is calculated as a price for 100$ swap
        resp = requests.get(url_0x, params=query,headers=header)
        swap = resp.json()
        time.sleep(1)
        spot_price = float(swap['price'])
        
        query['sellAmount'] = int(trade_value / tokens_market_data.loc['avalanche-2']['current_price'] * 10 ** decimals)
        resp = requests.get(url_0x, params=query,headers=header)
        swap = resp.json()
        time.sleep(1)
        del_price = float(swap['price'])
        
        slippage = del_price / spot_price - 1
        slippages.append({
            'id': id,
            'spot_price': spot_price,
            'delivery_price': del_price,
            'slippage': slippage
        })
    except KeyError:
        print(id)
        continue
    

havven
axelar
balancer
cartesi
allianceblock-nexera
ankreth
route
idia
realfevr
oath
ethos-reserve-note
granary
vnx-swiss-franc
build
soul-swap
spherium
shack
bios
nftearth


In [53]:
slippages_df = pd.DataFrame(slippages).sort_values('slippage',ascending=False).reset_index(drop=True)
slippages_df.head(20)

,id,spot_price,delivery_price,slippage
0,avalanche-2,1.000000e+00,1.000000e+00,0.000000
1,joe,4.709229e+01,4.616207e+01,-0.019753
2,avalaunch,3.598232e+01,3.330837e+01,-0.074313
3,benqi-liquid-staked-avax,8.946056e-01,8.207388e-01,-0.082569
4,benqi,2.666580e+03,2.424999e+03,-0.090596
5,pangolin,2.637987e+02,2.382475e+02,-0.096859
6,yield-yak,3.737580e-02,3.171731e-02,-0.151395
7,chainlink,1.422647e+00,1.096948e+00,-0.228938
8,colony,1.269639e+02,9.450437e+01,-0.255659
9,husky-avax,5.430466e+08,3.794110e+08,-0.301329


In [54]:
# Writing to database
slippages_to_save = slippages_df[['id','slippage']].head(20)
asset_columns = db_funcs.convert_to_sql_strings(list(slippages_to_save['id']))
slippage_values = list(slippages_to_save['slippage'])
db_funcs.insert_values(db_key_date,asset_columns,slippage_values,db_liquidity_table)


In [70]:
def check_avg_liquidity(date_delta):
    start_date = str(datetime.date.fromisoformat(db_key_date) - datetime.timedelta(days=date_delta))
    dataframe = pd.read_sql(f'Select * from {db_liquidity_table} where date >= ? ',db_funcs.create_connection(db_funcs.db),index_col='date',parse_dates=['date'],params=[start_date])
    dataframe = dataframe.ewm(span=3).mean().iloc[-1]
    dataframe = dataframe[dataframe > max_slippage]
    dataframe.index = db_funcs.convert_from_sql_strings(list(dataframe.index))
    return dataframe, list(dataframe.index)

avg_liq_dataframe ,include_list = check_avg_liquidity(liquidity_consistency)
avg_liq_dataframe


benqi-liquid-staked-avax   -0.082083
avalanche-2                 0.000000
joe                        -0.015494
Name: 2023-12-01 00:00:00, dtype: float64

In [71]:
qualified_assets = passed_q_and_non_q_checks.query('index in @include_list')
qualified_assets


,name,symbol,market_cap,market_cap_rank,current_price,total_supply,circulating_supply,addr
id,,,,,,,,
avalanche-2,Avalanche,AVAX,8.175898e+09,13.0,22.400000,4.332507e+08,3.652178e+08,0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7
benqi-liquid-staked-avax,BENQI Liquid Staked AVAX,SAVAX,1.676656e+08,232.0,25.010000,6.693711e+06,6.693711e+06,0x2b2c81e08f1af8835a78bb2a90ae924ace0ea4be
joe,JOE,JOE,1.618749e+08,240.0,0.474254,4.997094e+08,3.414594e+08,0x6e84a6216ea6dacc71ee8e6b0a5b7322eebc0fdd


### Marketcap ranking & filtering

In [72]:
chosen_tokens = qualified_assets.sort_values("market_cap", ascending=False)

## Weighting 

**Weighting requirements**
* The maximum weight any one token can have is 50%.
* All excess weight is proportionally redistributed to all uncapped tokens. 
* Any asset with a weight below 0.5% will be removed from the index.


### Weight mcaps

In [73]:
# exclude savax from weighting since it should be a part of avax weight
if 'benqi-liquid-staked-avax' in chosen_tokens.index:
    chosen_tokens_ex_savax = chosen_tokens.drop('benqi-liquid-staked-avax')
    weights = chosen_tokens_ex_savax['market_cap'].div(chosen_tokens_ex_savax['market_cap'].sum())
else:
    weights = chosen_tokens['market_cap'].div(chosen_tokens['market_cap'].sum())



### Adjust weights

This process adjusts weights to adhere to the weight constraints described in our methodology. In addition the `remove_tiny_weights` function checks for any assets with a weight below the minimum threshold of $0.5\%$ and proportionally increases all remaining weights (which are $> 0.5\%$). Note that after this procedure readjusting weights to respect maximum may be needed.

In [74]:
def adjust_weights(weights, max_weight):
    w = weights.copy()
    
    while (w > max_weight).any(axis=None):
        w[w > max_weight] = max_weight
        c = 1 - w.sum()
        w_less = w[w < max_weight]
        w[(w < max_weight) & (w > 0)] += w_less.div(w_less.sum()).mul(c)
    
    return w

In [75]:
def remove_tiny_weights(weights, min_weight):
    w = weights.copy()
    
    w[w < min_weight] = 0
    w = w.div(w.sum())
    
    return w

In [76]:
adjusted_weights = adjust_weights(weights, max_weight=max_weight) 
adjusted_weights = remove_tiny_weights(adjusted_weights, min_weight=min_weight)
adjusted_weights = adjust_weights(adjusted_weights, max_weight=max_weight)

### Split AVAX 

In [77]:
if 'avalanche-2' and 'benqi-liquid-staked-avax' in chosen_tokens.index:
    
    adjusted_weights['benqi-liquid-staked-avax'] = adjusted_weights['avalanche-2'] * avax_to_savax_split
    adjusted_weights['avalanche-2'] = adjusted_weights['avalanche-2'] * (1 - avax_to_savax_split)

adjusted_weights =  adjusted_weights.sort_values(ascending = False)


### Convert weights to the [1, 255] scale

On the contract side weights are integer numbers from the interval $[1, 255]$ with total sum $255$, so it's needed to convert retrieved weights to this format.

Note that one can't just round weights to integers after scaling, since it doesn't guarantee that their sum will be $255$. To fix that firstly floor function is applied to the weights and then $1$ is being added to the $k$ weights with largest fractional parts.

In [78]:
def convert_weights(weights):
    w_scaled = weights * 255
    w_res = np.floor(w_scaled).astype(int)    
    remainders = w_scaled - w_res    
    k = round(remainders.sum())
    w_below_max = w_res[w_res<128]
    for i in range(k):
        w_below_max[i] +=1
    for i in w_below_max.index:
        w_res[i] = w_below_max[i]
    return w_res

In [79]:
converted_weights = convert_weights(adjusted_weights)

/var/folders/4t/6j5ypx_s6r771shjr5dcct5w0000gn/T/ipykernel_79988/3538882253.py:8: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/var/folders/4t/6j5ypx_s6r771shjr5dcct5w0000gn/T/ipykernel_79988/3538882253.py:8: FutureWarning:

Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`



## Summary table

In [80]:
cai = pd.DataFrame()
cai.index = chosen_tokens.index
cai['name'] = chosen_tokens['name']
cai['symbol'] = chosen_tokens['symbol']
cai['market_cap'] = chosen_tokens['market_cap']
cai['price'] = chosen_tokens['current_price']
cai['weight'] = adjusted_weights
cai['weight_converted'] = converted_weights
cai['address'] = chosen_tokens['addr']

cai = cai[cai['weight'] > 0]
cai = cai.sort_values("market_cap", ascending=False)
cai

,name,symbol,market_cap,price,weight,weight_converted,address
id,,,,,,,
avalanche-2,Avalanche,AVAX,8.175898e+09,22.400000,0.2,51,0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7
benqi-liquid-staked-avax,BENQI Liquid Staked AVAX,SAVAX,1.676656e+08,25.010000,0.3,76,0x2b2c81e08f1af8835a78bb2a90ae924ace0ea4be
joe,JOE,JOE,1.618749e+08,0.474254,0.5,128,0x6e84a6216ea6dacc71ee8e6b0a5b7322eebc0fdd


In [81]:
# Add assets and weights to database
db_assets = db_funcs.convert_to_sql_strings(list(cai.index))
db_weights = list(cai['weight'])
db_funcs.insert_values(db_key_date,db_assets,db_weights,db_benchmark_table)

In [82]:
#Querying current contract to ascertain which assets will be dropped

cai_token_address = '0x48f88A3fE843ccb0b5003e70B4192c1d7448bEf0'
ia_contract = web3_object.eth.contract(address=cai_token_address,abi=index_anatomy)
addresses, weights = ia_contract.functions.anatomy().call()


for address in addresses:
    address = address.lower()
    if address not in list(cai['address']):
        data = tokens_market_data[tokens_market_data['addr']== address]
        cai.loc[data.index[0]] = [data['name'][0],data['symbol'][0],data['market_cap'][0],data['current_price'][0],0,0,address]

cai

,name,symbol,market_cap,price,weight,weight_converted,address
id,,,,,,,
avalanche-2,Avalanche,AVAX,8.175898e+09,22.400000,0.2,51,0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7
benqi-liquid-staked-avax,BENQI Liquid Staked AVAX,SAVAX,1.676656e+08,25.010000,0.3,76,0x2b2c81e08f1af8835a78bb2a90ae924ace0ea4be
joe,JOE,JOE,1.618749e+08,0.474254,0.5,128,0x6e84a6216ea6dacc71ee8e6b0a5b7322eebc0fdd


In [83]:
zipped_assets = list(zip(list(cai['address']),list(cai['weight_converted'])))
sorted_assets = sorted(zipped_assets,key = lambda x: int(x[0],base=0) )

asset_string = []
weight_string = []
for i in sorted_assets:
    asset_string.append(i[0])
    weight_string.append(f'{i[1]}')
asset_string = ','.join(asset_string)
weight_string = ','.join(weight_string)
print(asset_string)
print(weight_string)

0x2b2c81e08f1af8835a78bb2a90ae924ace0ea4be,0x6e84a6216ea6dacc71ee8e6b0a5b7322eebc0fdd,0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7
76,128,51
